In [37]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder 
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import StepLR 
from PIL import Image
from sklearn.metrics import f1_score

In [38]:
# Define the transform to resize images to (255, 255)
transform = transforms.Compose([
    transforms.Resize((255, 255)),
    transforms.ToTensor(),
    transforms.Grayscale()
])


train_path= "C:/Users/renax/Desktop/ACO/S9/Machine_Learning/plantvillage_deeplearning_paper_dataset/raw/grayscale_train"
test_path= "C:/Users/renax/Desktop/ACO/S9/Machine_Learning/plantvillage_deeplearning_paper_dataset/raw/grayscale_test"
# Define your dataset using ImageFolder and apply the transform
train_dataset = ImageFolder(root=train_path, transform=transform)
test_dataset = ImageFolder(root=test_path, transform=transform)


In [39]:

# Define the number of data loading workers
num_workers = 3  # You can adjust this number based on your CPU capacity

# Create data loaders with multiple workers
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=num_workers)



In [40]:

# Define a simple neural network model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 127 * 127, 128)
        self.fc2 = nn.Linear(128, 38)  # 38 output classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 16 * 127 * 127)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [41]:
# Create an instance of the model
model = SimpleModel()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss() 

optimizer = optim.SGD(
    model.parameters(),
    lr=0.005,  # Base learning rate
    momentum=0.9,
    weight_decay=0.0005,  # Weight decay
)

lr_scheduler = StepLR(optimizer, step_size=30, gamma=0.1)


In [42]:
epochs= 5

In [43]:

# Training loop (you can modify this based on your specific requirements)
for epoch in range(epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

 # Step the learning rate scheduler at the end of each epoch
    lr_scheduler.step()

    # Calculate accuracy on the test set (optional)
    correct = 0
    total = 0
    true_labels = []
    predicted_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            true_labels.extend(labels.tolist())
            predicted_labels.extend(predicted.tolist())

    accuracy = 100 * correct / total
    f1 = f1_score(true_labels, predicted_labels, average='weighted') 
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Test Accuracy: {accuracy}%, Test F1 Score: {f1}')

Epoch 1/5, Loss: 2.5750348567962646, Test Accuracy: 39.82642237222758%, Test F1 Score: 0.34725410053952
Epoch 2/5, Loss: 1.7169314622879028, Test Accuracy: 45.38733526197364%, Test F1 Score: 0.41957738353105034
